### Imports

In [1]:
from __future__ import print_function

%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

import SimpleITK as sitk
import cv2
import numpy as np
from os import listdir
from myshow import show_images, show_multimages

### Carga de imágenes y visualización

In [2]:
images = []
images2 = []
titles = []
for a in listdir("images"):
    im = cv2.imread("images/" + a)
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    images.append(im_gray)
    images2.append(sitk.ReadImage("images/" + a))
    titles.append(a)

show_images(images, titles=titles)

interactive(children=(IntSlider(value=3, description='N_imagen', max=5, min=1), Output()), _dom_classes=('widg…

### Creación automática de coordenadas semilla y rectángulo de búsqueda

In [3]:
seeds = []
seedmap = []
lungmap = []
rectangulos = []
Medidas = []

for im in images:
    # Binarizar imagen para crear máscara limpiar bordes y selección pulmones
    ret, thr = cv2.threshold(im, 125, 255, cv2.THRESH_BINARY)
    ret, thr3 = cv2.threshold(im, 185, 255, cv2.THRESH_BINARY)

    # Apertura asegurarse no borrar pulmon, creación máscara para contornos
    opened = cv2.morphologyEx(thr, cv2.MORPH_CLOSE, np.ones((15,15)))

    mask = opened == 0
    mask = np.uint8(mask)
    mask = mask * 255

    thr3 = mask + thr3
    opened2 = cv2.morphologyEx(thr3, cv2.MORPH_CLOSE, np.ones((2,15)))

    # Detección de contornos
    contours, hierarchy = cv2.findContours(opened2,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    # Cuadrado rodeando los contornos detectados
    x,y,w,h = cv2.boundingRect(contours[1])
    Rectangulo = im[y-20:y+h+20, x-20:x+w+20]
    
    rectangulos.append(Rectangulo)
    Medidas.append([x,y,w,h])
    
    # Dibujar máscara y hacer malla de semillas para el geodesic
    out = np.zeros(im.shape, dtype = "uint8")
    lungmap.append(out)
    mesh = out * 1
    mesh[::20,::20] = 1

    cv2.drawContours(out, contours, 1, (255,255,255), thickness=-1)
    cv2.drawContours(out, contours, 2, (255,255,255), thickness=-1)

    outmesh = mesh * out
    seedmap.append(outmesh)

    ix,iy = np.where(outmesh == 255)
    seeds.append([ix,iy])

show_multimages(images, lungmap, seedmap, rectangulos, titles=["images","lung_segment","seedmap", "rectangulos"])

interactive(children=(IntSlider(value=2, description='N_imagen', max=4, min=1), Output()), _dom_classes=('widg…

### Creación máscara segmentada

In [4]:
mask = []
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (16,16))
kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (29,29))

for im,med in zip(images,Medidas):
    # Binarizar imagen y filtrar con el rectángulo de los pulmones
    ret, thr = cv2.threshold(im, 205, 255, cv2.THRESH_BINARY_INV)
    ret, thr3 = cv2.threshold(im, 140, 1, cv2.THRESH_BINARY_INV)
    
    # Filtro de los laterales
    opened = cv2.morphologyEx(thr3, cv2.MORPH_OPEN, kernel)
    fil = opened == 0
    fil = np.uint8(fil)
    
    #Mask
    out = np.zeros(im.shape, dtype = "uint8")
    out[med[1]:med[1]+med[3], med[0]:med[0]+med[2]] = 1
    out = thr * fil * out
    
    out = cv2.morphologyEx(out, cv2.MORPH_OPEN, np.ones((2,11)))
    out = cv2.morphologyEx(out, cv2.MORPH_OPEN, np.ones((19,2)))
    out = cv2.morphologyEx(out, cv2.MORPH_CLOSE, kernel2)
    mask.append(sitk.GetImageFromArray(out))
    
show_images(mask)

interactive(children=(IntSlider(value=3, description='N_imagen', max=5, min=1), Output()), _dom_classes=('widg…

### Geodesic Active contour

In [5]:
contours = []
featureImages = []
levelsets = []
InitialImages = []
finals = []

for image, ele, ma in zip(images,seeds,mask):
    sigma = float(1)

    gradientMagnitude = sitk.GradientMagnitudeRecursiveGaussianImageFilter()
    gradientMagnitude.SetSigma(sigma)

    featureImage = sitk.BoundedReciprocal(gradientMagnitude.Execute(ma))

    seedImage = sitk.Image(ma.GetSize()[0],ma.GetSize()[1], sitk.sitkUInt8 )
    seedImage.SetSpacing(ma.GetSpacing() )
    seedImage.SetOrigin(ma.GetOrigin() )
    seedImage.SetDirection(ma.GetDirection() )
    
    for x,y in zip(ele[0],ele[1]):
        se = [float(y), float(x)]
        seedImage[ seedImage.TransformPhysicalPointToIndex(se) ] = 1

    distance = sitk.SignedMaurerDistanceMapImageFilter()
    distance.InsideIsPositiveOff()
    distance.UseImageSpacingOn()

    initialImage = sitk.BinaryThreshold( distance.Execute( seedImage ), -1000, 10 )
    initialImage = sitk.Cast( initialImage, featureImage.GetPixelID() ) * -1 + 0.5


    geodesicActiveContour = sitk.GeodesicActiveContourLevelSetImageFilter()
    geodesicActiveContour.SetPropagationScaling( 3 )
    geodesicActiveContour.SetCurvatureScaling( 7 )
    geodesicActiveContour.SetAdvectionScaling( 40 )
    geodesicActiveContour.SetMaximumRMSError( 0.001 )
    geodesicActiveContour.SetNumberOfIterations( 1000 )

    levelset = geodesicActiveContour.Execute( initialImage, featureImage )

    print( "RMS Change: ", geodesicActiveContour.GetRMSChange() )
    print( "Elapsed Iterations: ", geodesicActiveContour.GetElapsedIterations() )

    contour = sitk.BinaryContour(sitk.BinaryThreshold(levelset, -1000, 0), fullyConnected=True)
    
    # Listas para visualizar. Can be deleted except contours.
    contours.append(contour)
    levelsets.append(levelset)
    featureImages.append(featureImage)
    InitialImages.append(initialImage)
    finals.append(sitk.LabelOverlay(sitk.GetImageFromArray(image), contour))

show_multimages(InitialImages, featureImages, finals, levelsets, contours)

RMS Change:  0.11941580963603655
Elapsed Iterations:  1000
RMS Change:  0.041954754346236876
Elapsed Iterations:  1000
RMS Change:  0.0403315167705983
Elapsed Iterations:  1000
RMS Change:  0.11998947456263973
Elapsed Iterations:  1000
RMS Change:  0.0477235380590696
Elapsed Iterations:  1000


interactive(children=(IntSlider(value=3, description='N_imagen', max=5, min=1), Output()), _dom_classes=('widg…

### Probar con suavizado de imágenes

In [6]:
mask = []
mask2 = []
mask3 = []
for im in images:
    mas = cv2.blur(im,(45,45))
    mas2 = cv2.GaussianBlur(im, (9,215), 11)
    mas3 = cv2.medianBlur(im, 25)
    mask.append(sitk.GetImageFromArray(mas))
    mask2.append(sitk.GetImageFromArray(mas2))
    mask3.append(sitk.GetImageFromArray(mas3))

show_multimages(mask, mask2, mask3)

interactive(children=(IntSlider(value=2, description='N_imagen', max=3, min=1), Output()), _dom_classes=('widg…

### Geodesic Contour

In [7]:
contours = []
featureImages = []
levelsets = []
InitialImages = []
finals = []

for image, ele, ma in zip(images,seeds,mask2):
    sigma = float(1)

    gradientMagnitude = sitk.GradientMagnitudeRecursiveGaussianImageFilter()
    gradientMagnitude.SetSigma(sigma)

    featureImage = sitk.BoundedReciprocal(gradientMagnitude.Execute(ma))

    seedImage = sitk.Image(ma.GetSize()[0],ma.GetSize()[1], sitk.sitkUInt8 )
    seedImage.SetSpacing(ma.GetSpacing() )
    seedImage.SetOrigin(ma.GetOrigin() )
    seedImage.SetDirection(ma.GetDirection() )
    
    for x,y in zip(ele[0],ele[1]):
        se = [float(y), float(x)]
        seedImage[ seedImage.TransformPhysicalPointToIndex(se) ] = 1

    distance = sitk.SignedMaurerDistanceMapImageFilter()
    distance.InsideIsPositiveOff()
    distance.UseImageSpacingOn()

    initialImage = sitk.BinaryThreshold( distance.Execute( seedImage ), -1000, 10 )
    initialImage = sitk.Cast( initialImage, featureImage.GetPixelID() ) * -1 + 0.5


    geodesicActiveContour = sitk.GeodesicActiveContourLevelSetImageFilter()
    geodesicActiveContour.SetPropagationScaling( 3 )
    geodesicActiveContour.SetCurvatureScaling( 4.5 )
    geodesicActiveContour.SetAdvectionScaling( 45 )
    geodesicActiveContour.SetMaximumRMSError( 0.001 )
    geodesicActiveContour.SetNumberOfIterations( 1000 )

    levelset = geodesicActiveContour.Execute( initialImage, featureImage )

    print( "RMS Change: ", geodesicActiveContour.GetRMSChange() )
    print( "Elapsed Iterations: ", geodesicActiveContour.GetElapsedIterations() )

    contour = sitk.BinaryContour(sitk.BinaryThreshold(levelset, -1000, 0), fullyConnected=True)
    
    # Listas para visualizar. Can be deleted except contours.
    contours.append(contour)
    levelsets.append(levelset)
    featureImages.append(featureImage)
    InitialImages.append(initialImage)
    finals.append(sitk.LabelOverlay(sitk.GetImageFromArray(image), contour))

show_multimages(InitialImages, featureImages, finals, levelsets, contours)

RMS Change:  0.02629476442362858
Elapsed Iterations:  1000
RMS Change:  0.026831814795948438
Elapsed Iterations:  1000
RMS Change:  0.021374132424240216
Elapsed Iterations:  1000
RMS Change:  0.028283083427193826
Elapsed Iterations:  1000
RMS Change:  0.024633724743746824
Elapsed Iterations:  1000


interactive(children=(IntSlider(value=3, description='N_imagen', max=5, min=1), Output()), _dom_classes=('widg…